## Timeseries clustering

Time series clustering is to partition time series data into groups based on similarity or distance, so that time series in the same cluster are similar.

Methodology followed:
* Use Variational Recurrent AutoEncoder (VRAE) for dimensionality reduction of the timeseries
* To visualize the clusters, PCA and t-sne are used

Paper:
https://arxiv.org/pdf/1412.6581.pdf

#### Contents

0. [Load data and preprocess](#Load-data-and-preprocess)
1. [Initialize VRAE object](#Initialize-VRAE-object)
2. [Fit the model onto dataset](#Fit-the-model-onto-dataset)
3. [Transform the input timeseries to encoded latent vectors](#Transform-the-input-timeseries-to-encoded-latent-vectors)
4. [Save the model to be fetched later](#Save-the-model-to-be-fetched-later)
5. [Visualize using PCA and tSNE](#Visualize-using-PCA-and-tSNE)

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Import required modules

In [ ]:
from vrae.vrae import VRAE
from vrae.utils import *
import numpy as np
import torch

import plotly
from torch.utils.data import DataLoader, TensorDataset
plotly.offline.init_notebook_mode()

### Input parameters

In [ ]:
dload = './model_dir' #download directory

### Hyper parameters

In [62]:
hidden_size = 90
hidden_layer_depth = 1
latent_length = 20
batch_size = 32
learning_rate = 5e-3
n_epochs = 10
dropout_rate = 0.2
optimizer = 'Adam' # options: Adam, SGD
cuda = False # options: True, False
print_every=30
clip = True # options: True, False
max_grad_norm=5
loss = 'SmoothL1Loss' # options: SmoothL1Loss, MSELoss
block = 'LSTM' # options: LSTM, GRU

### Load data and preprocess

In [ ]:

X_train, X_val, y_train, y_val = open_data('data', ratio_train=0.9)

num_classes = len(np.unique(y_train))
base = np.min(y_train)  # Check if data is 0-based
if base != 0:
    y_train -= base
y_val -= base

'''
signals = np.load('data/signals.npy')
signals = signals/(np.max(np.abs(signals),axis=0))
signals = signals[0:-1:5, :]
signals = np.expand_dims(signals.T, -1)
N = signals.shape[0]
ratio_train = 0.8
ind_cut = int(ratio_train * N)
ind = np.random.permutation(N)
X_train, X_val = signals[ind[:ind_cut], :, :], signals[ind[ind_cut:], :, :]
y_val=np.zeros([1010,1,1])
dataset_val = np.zeros([5046,1,1])
'''

In [ ]:
train_dataset = TensorDataset(torch.from_numpy(X_train))
test_dataset = TensorDataset(torch.from_numpy(X_val))

# dataset = TensorDataset(torch.from_numpy(signals))

In [ ]:
dataset.tensors[0].shape

**Fetch `sequence_length` from dataset**

In [ ]:
sequence_length = X_train.shape[1]

**Fetch `number_of_features` from dataset**

This config corresponds to number of input features

In [ ]:
number_of_features = X_train.shape[2]

### Initialize VRAE object

VRAE inherits from `sklearn.base.BaseEstimator` and overrides `fit`, `transform` and `fit_transform` functions, similar to sklearn modules

In [63]:
vrae = VRAE(sequence_length=sequence_length,
            number_of_features = number_of_features,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate,
            optimizer = optimizer, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            loss = loss,
            block = block,
            dload = dload)

### Fit the model onto dataset

In [30]:
vrae.fit(train_dataset)

#If the model has to be saved, with the learnt parameters use:
# vrae.fit(dataset, save = True)

Epoch: 0
Batch 30, loss = 1499.0876, recon_loss = 1499.0282, kl_loss = 0.0595
Batch 60, loss = 1217.2352, recon_loss = 1213.0627, kl_loss = 4.1725
Batch 90, loss = 938.2387, recon_loss = 934.0878, kl_loss = 4.1509
Batch 120, loss = 1022.1292, recon_loss = 1018.2272, kl_loss = 3.9019
Batch 150, loss = 952.3290, recon_loss = 948.8663, kl_loss = 3.4627
Batch 180, loss = 1046.6816, recon_loss = 1044.2013, kl_loss = 2.4804
Batch 210, loss = 928.9993, recon_loss = 926.8914, kl_loss = 2.1080
Batch 240, loss = 967.8495, recon_loss = 965.8349, kl_loss = 2.0146
Average loss: 1082.0877
Epoch: 1
Batch 30, loss = 976.8624, recon_loss = 975.2009, kl_loss = 1.6614


KeyboardInterrupt: 

### Transform the input timeseries to encoded latent vectors

In [ ]:
z_run = vrae.transform(test_dataset)
dataset_run = vrae.transform(dataset)

#If the latent vectors have to be saved, pass the parameter `save`
# z_run = vrae.transform(dataset, save = True)

### Save the model to be fetched later

In [ ]:
vrae.save('vrae_10.pth')

# To load a presaved model, execute:
# vrae.load('vrae.pth')

### Visualize using PCA and tSNE

In [ ]:
# y_val = np.ones(test_dataset.tensors[0].shape[0])
# plot_clustering(z_run, y_val, engine='matplotlib', download = False)
plot_clustering(dataset_run, dataset_val, engine='matplotlib', download = False)
# If plotly to be used as rendering engine, uncomment below line
#plot_clustering(z_run, y_val, engine='plotly', download = False)

In [ ]:
z_run.shape